In [23]:
import psycopg2

from dotenv import load_dotenv
import os
import time

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras

import joblib

In [24]:

# load scaler
scalerx = joblib.load("../models/scalerx.save")
scalery = joblib.load("../models/scalery.save")

# load model
model = keras.models.load_model("../models/nn_model.h5")

In [25]:
# load dotenv
load_dotenv()

# connect to database
conn = psycopg2.connect(
    host=os.getenv("HOST"),
    database=os.getenv("DBNAME"),
    user=os.getenv("DBUSER"),
    password=os.getenv("PASS"),
    port=os.getenv("PORT")
)

# create cursor
cursor = conn.cursor()

In [26]:
# get today 
today = time.strftime("%Y-%m-%d")

In [27]:
query = """
            SELECT * FROM one_hour_candle
            WHERE symbol = %s AND bucket >= NOW() - INTERVAL '40 days'
            ORDER BY bucket;
"""

symbol = "AAPL"
cursor.execute(query, (symbol,))
results = cursor.fetchall()
print(results)

[(datetime.datetime(2023, 6, 7, 12, 0, tzinfo=datetime.timezone.utc), 'AAPL', 179.1, 179.17, 177.69, 178.88, None), (datetime.datetime(2023, 6, 7, 13, 0, tzinfo=datetime.timezone.utc), 'AAPL', 178.87, 181.17999268, 178.03, 180.43, 10010869), (datetime.datetime(2023, 6, 7, 14, 0, tzinfo=datetime.timezone.utc), 'AAPL', 180.42, 180.64, 178.065, 178.72, 19680567), (datetime.datetime(2023, 6, 7, 15, 0, tzinfo=datetime.timezone.utc), 'AAPL', 178.71, 179.6, 177.82001, 178.88, 25692532), (datetime.datetime(2023, 6, 7, 16, 0, tzinfo=datetime.timezone.utc), 'AAPL', 178.91, 179.04, 178.01, 178.105, 30732127), (datetime.datetime(2023, 6, 7, 17, 0, tzinfo=datetime.timezone.utc), 'AAPL', 178.13, 178.4677, 177.33, 177.81, 36655760), (datetime.datetime(2023, 6, 7, 18, 0, tzinfo=datetime.timezone.utc), 'AAPL', 177.795, 178.28, 177.61, 178.25, 40793075), (datetime.datetime(2023, 6, 7, 19, 0, tzinfo=datetime.timezone.utc), 'AAPL', 178.24, 178.4999, 177.4714, 177.9605, 50053715), (datetime.datetime(2023, 

In [28]:
ts = pd.DataFrame(results, columns=["bucket", "symbol", "open", "high", "low", "close", "volume"])
ts["bucket"] = pd.to_datetime(ts["bucket"])
ts.set_index("bucket", inplace=True)

In [29]:
len(ts['close'])

278

In [30]:
# create dataset X and y, rolling window of 7 days
df = ts['close']
# reverse
# df = df.iloc[::-1]

time_steps_past = 6*24 # 6 days * 24 hours
X = []

for i in range(time_steps_past, len(df)):
    X.append(df[i-time_steps_past:i])

X = np.array(X)

In [31]:
# scale
X1 = scalerx.transform(X)

In [32]:
# predict
y_pred = model.predict(X)

# inverse transform
y_pred = scalery.inverse_transform(y_pred)

# return last prediction
print(y_pred[-1][0])

5/5 [==============================] - 1s 17ms/step
2943.4827
